In [5]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)
from sklearn.preprocessing import LabelEncoder
from imblearn.combine import SMOTEENN
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns



# =====================================================
# 1. DATA PREPROCESSING
# =====================================================
def preprocess_data(df, dataset_name):
    df = df.copy()
    le = LabelEncoder()

    if dataset_name == 'adult':
        df.replace("?", np.nan, inplace=True)
        df.dropna(inplace=True)
        df["income"] = df["income"].apply(lambda x: 1 if ">50K" in x else 0)

        y = df["income"].values
        s = df["sex"].apply(lambda x: 1 if x.strip() == "Male" else 0).values
        df.drop(["income", "sex"], axis=1, inplace=True)

    elif dataset_name == "propublica":
        y = df["Two_yr_Recidivism"].values
        s = df["African_American"].values

        race_columns = ["African_American", "Asian", "Hispanic", "Native_American", "Other"]
        df.drop(["Two_yr_Recidivism"] + race_columns, axis=1, inplace=True)

    elif dataset_name == "framingham":
        df.dropna(inplace=True)

        y = df["TenYearCHD"].values
        s = df["male"].values
        df.drop(["TenYearCHD", "male"], axis=1, inplace=True)

    # Encode categorical variables
    for col in df.columns:
        if df[col].dtype == "object" or isinstance(df[col].iloc[0], str):
            df[col] = le.fit_transform(df[col].astype(str))

    X = df.values
    return X, y, s



# =====================================================
# 2. BMNB MODEL CLASS
# =====================================================
class BMNB:
    def __init__(self, alpha=0.5, class_priors=[0.5, 0.5]):
        self.alpha = alpha
        self.class_priors = class_priors
        self.global_model = GaussianNB(priors=self.class_priors)
        self.group_models = {}
        self.thresholds = {}

    def fit(self, X, y, s):
        self.groups = np.unique(s)

        # Global NB model
        self.global_model.fit(X, y)

        # Group-specific NB models
        for g in self.groups:
            idx = (s == g)
            g_model = GaussianNB(priors=self.class_priors)
            g_model.fit(X[idx], y[idx])
            self.group_models[g] = g_model

    def predict_proba_blended(self, X, s):
        global_probs = self.global_model.predict_proba(X)[:, 1]
        blended = np.zeros(len(X))

        for g in self.groups:
            idx = (s == g)
            g_probs = self.group_models[g].predict_proba(X[idx])[:, 1]
            blended[idx] = self.alpha * global_probs[idx] + (1 - self.alpha) * g_probs

        return blended

    def calibrate_thresholds(self, y_true, probs, s):
        self.thresholds = {}
        target_rate = np.mean(y_true)

        for g in self.groups:
            g_probs = probs[s == g]
            if len(g_probs) == 0:
                self.thresholds[g] = 0.5
                continue

            sorted_probs = np.sort(g_probs)
            idx = int((1 - target_rate) * len(sorted_probs))
            self.thresholds[g] = sorted_probs[-idx] if idx > 0 else 1.0

    def predict_with_thresholds(self, probs, s):
        preds = np.zeros_like(probs, dtype=int)

        for g in self.groups:
            idx = (s == g)
            preds[idx] = (probs[idx] >= self.thresholds[g]).astype(int)

        return preds



# =====================================================
# 3. EVALUATION HELPERS
# =====================================================
def compute_metrics(y_test, y_pred, s_test):
    groups = np.unique(s_test)
    metrics = {}

    for g in groups:
        idx = (s_test == g)
        yt, yp = y_test[idx], y_pred[idx]

        TP = np.sum((yt == 1) & (yp == 1))
        TN = np.sum((yt == 0) & (yp == 0))
        FP = np.sum((yt == 0) & (yp == 1))
        FN = np.sum((yt == 1) & (yp == 0))

        TPR = TP / (TP + FN) if (TP + FN) > 0 else 0
        FPR = FP / (FP + TN) if (FP + TN) > 0 else 0
        PPR = np.mean(yp)

        metrics[g] = {"TPR": TPR, "FPR": FPR, "PPR": PPR}

    g0, g1 = metrics[0], metrics[1]

    EOD = g1["TPR"] - g0["TPR"]
    EMOD = g1["FPR"] - g0["FPR"]
    SPD = g1["PPR"] - g0["PPR"]
    DI = g1["PPR"] / g0["PPR"] if g0["PPR"] > 0 else 0

    BI = (abs(EOD) + abs(EMOD) + abs(SPD) + abs(1 - DI)) / 4
    FS = 1 - BI

    return {
        "EOD": EOD, "EMOD": EMOD, "SPD": SPD, "DI": DI,
        "BI": BI, "FS": FS
    }



# =====================================================
# 4. ABLATION STUDY RUNNER
# =====================================================
def run_ablation(X, y, s, dataset_name, alpha=0.5):
    # STEP 1: SMOTEENN
    sm = SMOTEENN(random_state=42)
    X_r, y_r = sm.fit_resample(X, y)

    # STEP 2: Nearest Neighbor match sensitive attribute
    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(X)
    _, idx = nn.kneighbors(X_r)
    s_r = s[idx.flatten()]

    # STEP 3: Train-test split
    X_train, X_test, y_train, y_test, s_train, s_test = train_test_split(
        X_r, y_r, s_r, test_size=0.2, random_state=42, stratify=y_r
    )

    #  =============================
    #  ABLATION 1: BLENDED ONLY
    #  =============================
    model1 = BMNB(alpha=alpha)
    model1.fit(X_train, y_train, s_train)
    probs1 = model1.predict_proba_blended(X_test, s_test)
    y_pred1 = (probs1 >= 0.5).astype(int)

    #  =============================
    #  ABLATION 2: THRESHOLD ONLY
    #  =============================
    model2 = BMNB(alpha=0)   # NO blending, use global model only
    model2.fit(X_train, y_train, s_train)
    probs2 = model2.global_model.predict_proba(X_test)[:, 1]
    model2.calibrate_thresholds(y_test, probs2, s_test)
    y_pred2 = model2.predict_with_thresholds(probs2, s_test)

    #  =============================
    #  FULL BMNB (BLENDING + THRESHOLDING)
    #  =============================
    model3 = BMNB(alpha=alpha)
    model3.fit(X_train, y_train, s_train)
    probs3 = model3.predict_proba_blended(X_test, s_test)
    model3.calibrate_thresholds(y_test, probs3, s_test)
    y_pred3 = model3.predict_with_thresholds(probs3, s_test)

    # PACKAGE RESULTS
    results = {
        "blended_only": {
            "acc": accuracy_score(y_test, y_pred1),
            "fairness": compute_metrics(y_test, y_pred1, s_test),
        },
        "threshold_only": {
            "acc": accuracy_score(y_test, y_pred2),
            "fairness": compute_metrics(y_test, y_pred2, s_test),
        },
        "full_bmnb": {
            "acc": accuracy_score(y_test, y_pred3),
            "fairness": compute_metrics(y_test, y_pred3, s_test),
        }
    }

    return results



# =====================================================
# 5. RUN FOR A DATASET
# =====================================================
# Example:
# df = pd.read_csv("adult.csv")
# X, y, s = preprocess_data(df, "adult")
# results = run_ablation(X, y, s, "adult")
# print(results)

# framingham_df = pd.read_csv("framingham.csv")
# X, y, s = preprocess_data(framingham_df, 'framingham')
# results = run_ablation(X, y, s, 'framingham')
# print(results)

# propublica_df = pd.read_csv("propublica_data_for_fairml.csv")
# X, y, s = preprocess_data(propublica_df, 'propublica')
# results = run_ablation(X, y, s, 'propublica')
# print(results)


adult_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"

column_names = [
    "age", "workclass", "fnlwgt", "education", "education-num",
    "marital-status", "occupation", "relationship", "race", "sex",
    "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"
]

# Load and assign column names
adult_df = pd.read_csv(adult_url, header=None, names=column_names, skipinitialspace=True)
X, y, s = preprocess_data(adult_df, "adult")
results = run_ablation(X, y, s, 'adult')
print(results)

{'blended_only': {'acc': 0.6697163769441903, 'fairness': {'EOD': -0.03295188012426081, 'EMOD': 0.0006703922050274314, 'SPD': 0.13256528481568633, 'DI': 1.685746590144695, 'BI': 0.2129835368224174, 'FS': 0.7870164631775826}}, 'threshold_only': {'acc': 0.7156907593778591, 'fairness': {'EOD': -0.22541425378037538, 'EMOD': -0.14405244411692458, 'SPD': 5.2568162230981486e-06, 'DI': 1.000012789660569, 'BI': 0.09237118609352303, 'FS': 0.907628813906477}}, 'full_bmnb': {'acc': 0.7143183897529735, 'fairness': {'EOD': -0.21732800041110323, 'EMOD': -0.1481285310734463, 'SPD': 5.2568162230981486e-06, 'DI': 1.000012789660569, 'BI': 0.09136864449033542, 'FS': 0.9086313555096646}}}
